# Import

In [181]:
%matplotlib qt
%gui qt

import re
import os
import sys

import numpy as np
from loguru import logger

import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

import pyqtgraph as pg
import pyqtgraph.opengl as gl

from collections import defaultdict
from datetime import datetime

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.interpolate as interp

In [165]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [166]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,1] * np.cos(np.deg2rad(_data[:,2])) 
        _Zimag  = _data[:,1] * np.sin(np.deg2rad(_data[:,2])) 
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

In [167]:


def EIS_recal_ver02(data, _phz_0 = None):
    f_poi = data[0,:]
    # Z_poi = data[1,:] * np.exp(1j*np.deg2rad(data[2,:]))
    Z_poi = data[1,:] + 1j*data[2,:]
    Y_poi = 1/Z_poi

    Rg0 = 1.611e13
    Cp0 = 1.4e-9
    
    _Rg0_rescale = 1/Rg0*np.power(f_poi,1.583)
    _Cp0_rescale = Cp0*np.power(f_poi,0.911)
    Y_org = Y_poi - _Rg0_rescale + 1j*_Cp0_rescale
    # Y_org = Y_poi - _Rg0_rescale 
    # Y_org = Y_poi + 1j*_Cp0_rescale
    # Y_org = Y_poi
    Z_org = 1/Y_org

    # Phz Calibration
    if _phz_0 is None:
        _phz_0 = np.loadtxt("./phz_Calib.txt")
    
    Z_ampC = np.abs(Z_org)
    # Z_phzC = np.angle(Z_org) - _phz_0
    Z_phzC = np.angle(Z_org) - _phz_0

    Z_rec = Z_ampC * np.exp(1j*Z_phzC)

    # C = 5e-10
    Rs0 = 100
    Z_rec = Z_rec - Rs0



    Cp0 = 5e-10
    _Cp0_rescale = Cp0 * f_poi
    Z_rec = 1/(1/Z_rec - 1j * _Cp0_rescale)

    

    # Ls0 = 1.7e-4
    Ls0 = 5e-4
    _Ls0_rescale = Ls0 * f_poi
    Z_rec = Z_rec - 1j * _Ls0_rescale

    # C = 5e-10
    Rs0 = 566
    Z_rec = Z_rec - Rs0
    
    return np.stack([f_poi, np.real(Z_rec), np.imag(Z_rec)], axis=1).T
    

## Data Read-in

In [168]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
ch_id = 13    # Up & Down, 3 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

# rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
# ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/06017758_归档"
# ch_id = 96     # Perfect of Perfect

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run With Error

# freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)
freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
EISDict = gatherCSV(rootPath)
chData = readChannel(ch_id, EISDict)

if True:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData)[0]):
        ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        chData[i,:,:] = ch_eis
chData = chData[:,:,freq_list]

# chData = chData[:,:,91:100]


np.shape(chData)
        

2025-04-03 20:00:34.165 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241022_01
2025-04-03 20:00:34.166 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-03 20:00:34.166 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-03 20:00:34.167 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-03 20:00:34.168 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-03 20:00:34.168 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241024_01
2025-04-03 20:00:34.169 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-03 20:00:34.169 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-03 20:00:34.170 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-03 20:00:34.170 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-03 20:00:34.171 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241028_01
2025-04-03 20:00:34.171 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-03 20:00:34.172 | INFO    

(13, 3, 101)

## Data Cleaning

In [ ]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from  Outlier import OutlierDetection

CLEAN_FLAG = True
if CLEAN_FLAG:
    eis_seq, eis_cluster, eis_anomaly, leaf_anomaly = OutlierDetection.OutlierDetection(chData)
else: 
    eis_seq = np.arange(np.shape(chData)[0])

## Data Plot

In [198]:
if False:
    fig, axis = plt.subplots(1,4,figsize=(15,6))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(np.shape(chData)[0]):
    # for i in [1]:
        ch_eis = chData[i,:,:]
        # ch_eis = EIS_recal(chData[i,:,:].T).T
        # ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        _color = cmap(i/np.shape(chData)[0])
        axis[0].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[1].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
        # axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    
        # _poi_Z = np.log(np.abs(ch_eis[1,:]+1j*ch_eis[2,:]))
        # _poi_P = np.angle(ch_eis[1,:]+1j*ch_eis[2,:])
        # _poi_eis = _poi_Z * np.exp(1j*_poi_P)
        # axis[3].plot(np.real(_poi_eis), -np.imag(_poi_eis), color = _color, linewidth=2, label=f"Session {i}")
        _poi_Z = np.log(ch_eis[1,:]+1j*ch_eis[2,:])
        axis[3].plot(np.real(_poi_Z), -np.imag(_poi_Z), color = _color, linewidth=2, label=f"Session {i}")
        

# axis[0].legend(frameon=False, loc='upper left')

In [199]:
fig= plt.figure(figsize=(15,8), constrained_layout=False)
axis = [0] * 8
axis[0] = fig.add_subplot(2,4,1, projection='3d')   
axis[1] = fig.add_subplot(2,4,2)            
axis[2] = fig.add_subplot(2,4,3)         
axis[3] = fig.add_subplot(2,4,4)      
axis[4] = fig.add_subplot(2,4,5, projection='3d')      
axis[5] = fig.add_subplot(2,4,6)         
axis[6] = fig.add_subplot(2,4,7)         
axis[7] = fig.add_subplot(2,4,8)    

init_elev = 21  # 仰角
init_azim = 55  # 方位角
axis[0].view_init(elev=init_elev, azim=init_azim)
axis[4].view_init(elev=init_elev, azim=init_azim)


num_samples = np.shape(chData)[0]

_x = np.arange(num_samples)[eis_seq]
_y = np.log10(chData[0,0,:]).flatten()
X, Y = np.meshgrid(_x, _y, indexing='ij')
axis[0].plot_surface(X, Y, np.log10(np.abs(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)
axis[4].plot_surface(X, Y, np.rad2deg(np.angle(chData[eis_seq,1,:]+1j*chData[eis_seq,2,:])), cmap='viridis_r', alpha=0.8)



cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(len(eis_seq)):
    _x = eis_seq[i]
    ch_eis = chData[_x,:,:]
    _color = cmap(_x/num_samples)
    axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{i:02d}")
    axis[5].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{i:02d}")


cmap = plt.colormaps.get_cmap('Set1')
for i in range(len(eis_seq)):
    _x = eis_seq[i]
    ch_eis = chData[_x,:,:]
    _color = cmap(eis_cluster[i])
    axis[2].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")
    axis[6].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"{chr(ord('A')+eis_cluster[i])}")

_legend_handle = []
for i in range(len(np.unique(eis_cluster))):
    _legend_handle.append(mpatches.Patch(color = cmap(i), label = f"{chr(ord('A')+i)}:{len(eis_cluster[eis_cluster==i])}"))
axis[2].legend(handles=_legend_handle)

axis[2].sharex(axis[1])
axis[6].sharex(axis[5])


cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(len(eis_anomaly)):
    _x = eis_anomaly[i]
    ch_eis = chData[_x,:,:]
    _color = cmap(_x/num_samples)
    axis[3].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"S{_x:02d}")
    axis[7].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"S{_x:02d}")
axis[3].legend()
axis[3].sharex(axis[1])
axis[7].sharex(axis[5])



## Input Layer

In [172]:
def DataLoader(chData, eis_seq, eis_cluster, SPEED_RATE = 1):
    # Speed Rate = 10 means 1 day = 10 points
    x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    x_day = [x_day[i] for i in eis_seq]

    x_train = np.array([(poi - x_day[0]).days for poi in x_day])
    x_eval = np.linspace(0,max(x_train),max(x_train)*SPEED_RATE+1)

    # chData_log = np.log(chData[eis_seq,1,:] + 1j*chData[eis_seq,2,:])
    # y_train = np.stack([chData_log.real.T,chData_log.imag.T], axis=2)
    chData_lin = chData[eis_seq,1,:] + 1j*chData[eis_seq,2,:]
    y_train = np.stack([chData_lin.real.T,chData_lin.imag.T], axis=2)


    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)}")

    return x_train, y_train, x_eval


# Interpolation

In [237]:
from scipy.interpolate import CubicSpline

def weighted_piecewise_interp(x, y, x_new, states = None):
    if states is None:
        states = np.zeros_like(x)
    unique_states = np.unique(states)
    y_new = np.zeros((np.shape(x_new)[0], np.shape(y)[1]))

    for i in range(len(unique_states) - 1):
        # 取当前状态和下一个状态的数据
        state_mask = (states == unique_states[i])
        next_state_mask = (states == unique_states[i + 1])
        
        x_state = x[state_mask]
        y_state = y[state_mask]
        x_next_state = x[next_state_mask]
        y_next_state = y[next_state_mask]
        # 计算当前 state 的插值
        cs = CubicSpline(x_state, y_state, bc_type='clamped')

        # 对新插值点进行计算
        mask_new = (x_new >= x_state.min()) & (x_new <= x_state.max())
        y_new[mask_new] = cs(x_new[mask_new])

        # 处理状态之间的插值
        trans_mask = (x_new > x_state.max()) & (x_new < x_next_state.min())
        if np.any(trans_mask):
            # 计算过渡区的线性权重
            alpha = (x_new[trans_mask] - x_state.max()) / (x_next_state.min() - x_state.max())
            y_new[trans_mask] = (1 - alpha) * cs(x_state.max()) + alpha * y_next_state[0]

    return y_new

# # 示例数据
# x_train = np.array([0, 1, 2, 4, 5, 7, 8, 10])
# x_eval = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# # 计算 y_train，其中每一列代表一个不同的相位（phi）
# # phis = np.array([0, np.pi/6, np.pi/3, np.pi/2])
# phis = np.array([np.pi * i/8 for i in range(8)])
# y_train = np.array([[np.sin(2 * np.pi * (x / 10) + phi) for phi in phis] for x in x_train])

# y_train[5:] = y_train[5:]/3

# # 初始化 y_eval，形状为 (len(x_eval), len(phis))
# y_eval = np.zeros((len(x_eval), len(phis)))

# cs = CubicSpline(x_train, y_train, bc_type='natural')
# y_eval = cs(x_eval)

# fig = plt.figure(figsize=(8,8))
# axis1 = fig.add_subplot(1,1,1, projection = '3d')

# _x = np.arange(np.shape(x_eval)[0])
# _y = np.arange(np.shape(phis)[0])
# X, Y = np.meshgrid(_x, _y, indexing='ij')
# axis1.plot_surface(X, Y, y_eval, cmap='viridis_r', alpha=0.8)



## Run

## Simple CubicSpline

In [240]:
if True:
    # not-a-knot
    # clamped
    x_train, y_train, x_eval = DataLoader(chData, eis_seq, eis_cluster, SPEED_RATE=1)
    y_real_cs = CubicSpline(x_train, y_train[:,:,0].T, bc_type='not-a-knot')
    y_imag_cs = CubicSpline(x_train, y_train[:,:,1].T, bc_type='not-a-knot')
    y_eval = np.stack((y_real_cs(x_eval).T, y_imag_cs(x_eval).T), axis=2)
    y_EIS = y_eval[:,:,0] + 1j*y_eval[:,:,1]
    y_org = y_train[:,:,0] + 1j*y_train[:,:,1]


2025-04-03 20:36:29.765 | INFO     | __main__:DataLoader:15 - 
x: (11,) 
y: (101, 11, 2) 
x_pred(19,)


## weighted_piecewise_interp

In [241]:
# x_train, y_train, x_eval = DataLoader(chData, eis_seq, eis_cluster, SPEED_RATE=1)
# y_eval_real = weighted_piecewise_interp(x_train, y_train[:,:,0].T, x_eval, eis_cluster).T
# y_eval_imag = weighted_piecewise_interp(x_train, y_train[:,:,1].T, x_eval, eis_cluster).T
# y_eval = np.stack((y_eval_real, y_eval_imag),axis=2)
# y_EIS = y_eval[:,:,0] + 1j*y_eval[:,:,1]
# y_org = y_train[:,:,0] + 1j*y_train[:,:,1]


In [242]:
np.shape(y_eval)

(101, 19, 2)

## Plot Result

In [246]:
fig= plt.figure(figsize=(15,8), constrained_layout=False)
axis = [0] * 8
axis[0] = fig.add_subplot(2,3,1, projection='3d')   
axis[1] = fig.add_subplot(2,3,2)            
axis[2] = fig.add_subplot(2,3,3)      
axis[3] = fig.add_subplot(2,3,4, projection='3d')      
axis[4] = fig.add_subplot(2,3,5)         
axis[5] = fig.add_subplot(2,3,6)    

init_elev = 21  # 仰角
init_azim = 55  # 方位角
axis[0].view_init(elev=init_elev, azim=init_azim)
axis[3].view_init(elev=init_elev, azim=init_azim)



_y = np.arange(np.shape(x_eval)[0])
_x = np.log10(chData[0,0,:]).flatten()
X, Y = np.meshgrid(_x, _y, indexing='ij')
axis[0].plot_surface(X, Y, np.log10(np.abs(y_EIS)), cmap='viridis_r', alpha=0.8)
axis[3].plot_surface(X, Y, np.rad2deg(np.angle(y_EIS)), cmap='viridis_r', alpha=0.8)


cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(np.shape(y_EIS)[1]):
    _color = cmap(i/np.shape(x_eval)[0])
    axis[1].loglog(ch_eis[0,:], np.abs(y_EIS[:,i]), color = _color, linewidth=2, label=f"S{i:02d}")
    axis[4].semilogx(ch_eis[0,:], np.rad2deg(np.angle(y_EIS[:,i])), color = _color, linewidth=2, label=f"S{i:02d}")


# cmap = plt.colormaps.get_cmap('viridis_r')
# for i in range(np.shape(y_EIS)[0]):
#     _color = cmap(i/np.shape(x_eval)[0])
#     axis[2].plot(x_eval, np.abs(y_EIS[i,:]), color = _color, linewidth=2, label=f"S{i:03d}")
#     axis[2].plot(x_train, np.abs(y_org[i,:]), 'r.')
#     axis[5].plot(x_eval, np.rad2deg(np.angle(y_EIS[i,:])), color = _color, linewidth=2, label=f"S{i:03d}")
#     axis[5].plot(x_train, np.rad2deg(np.angle(y_org[i,:])), 'r.')
# axis[5].set_ylim([-90,20])

cmap = plt.colormaps.get_cmap('viridis_r')
for i in range(np.shape(y_EIS)[0]):
# for i in range(40,50):
    _color = cmap(i/np.shape(x_eval)[0])
    axis[2].semilogy(x_eval, np.real(y_EIS[i,:]), color = _color, linewidth=2, label=f"S{i:03d}")
    axis[2].semilogy(x_train, np.real(y_org[i,:]), 'r.')
    axis[5].semilogy(x_eval, -np.imag(y_EIS[i,:]), color = _color, linewidth=2, label=f"S{i:03d}")
    axis[5].semilogy(x_train, -np.imag(y_org[i,:]), 'r.')



